In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
import random
import re


def crawl_foi_decisions(session: requests.Session, url: str, keyword: str) -> pd.DataFrame:
    """
    爬取金融消費評議中心(FOI)的評議決定書列表。

    本函數會模擬使用者在網站上進行查詢的操作，
    首先發送一個 GET 請求以獲取表單需要的驗證資訊 (VIEWSTATE)，
    然後帶上查詢條件發送 POST 請求來取得搜尋結果。
    最後，解析結果頁面的 HTML，並將資料整理成 Pandas DataFrame。

    Args:
        session (requests.Session): 用於保持連線狀態的 Session 物件。
        url (str): 要爬取的目標網站 URL。
        keyword (str): 內容檢索的關鍵字。

    Returns:
        pd.DataFrame: 包含評議案件資訊的 DataFrame，
                      欄位包括 '評議字號', '評議決定日期', '爭議類型', '下載連結'。
                      如果爬取失敗或沒有找到結果，則返回一個空的 DataFrame。
    """
    try:
        # --- 步驟 1: 發送 GET 請求以獲取初始表單資訊 ---
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        res_get = session.get(url, headers=headers, timeout=15)
        res_get.raise_for_status()

        soup_get = BeautifulSoup(res_get.text, 'html.parser')

        viewstate = soup_get.find(id='__VIEWSTATE').get('value')
        viewstategenerator = soup_get.find(
            id='__VIEWSTATEGENERATOR').get('value')

        # --- 步驟 2: 準備 POST 請求的表單資料 (payload) ---
        form_data = {
            '__EVENTTARGET': '',
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': viewstate,
            '__VIEWSTATEGENERATOR': viewstategenerator,
            'Foi$cph_content$ddlTypeID': 'I03',
            'Foi$cph_content$ddl_page': '100',
            'Foi$cph_content$ddl_ResultKind': 'I03001',
            'Foi$cph_content$hResName': '申請人主張有理由',
            'Foi$cph_content$ddlVerticals': '0',
            'Foi$cph_content$ddlSubVerticals': '0',
            'Foi$cph_content$dll_ControversyKind': '0',
            'Foi$cph_content$txt_BYear': '',
            'Foi$cph_content$dll_BCase': '評',
            'Foi$cph_content$txt_Bno': '',
            'Foi$cph_content$txt_Content': keyword,
            'Foi$cph_content$txt_Syear': '',
            'Foi$cph_content$txt_Smonth': '',
            'Foi$cph_content$txt_Sday': '',
            'Foi$cph_content$txt_Eyear': '',
            'Foi$cph_content$txt_Emonth': '',
            'Foi$cph_content$txt_Eday': '',
            'Foi$cph_content$btn_submit': '送出查詢',
            'Foi$cph_content$ddlOrderBy': '1',  # 依照日期由新到舊排序
            'Foi$cph_content$hSort': '1',
        }

        # --- 步驟 3: 發送 POST 請求以取得查詢結果 ---
        res_post = session.post(url, data=form_data,
                                headers=headers, timeout=15)
        res_post.raise_for_status()

        # --- 步驟 4: 解析查詢結果頁面 ---
        soup_post = BeautifulSoup(res_post.text, 'html.parser')
        result_div = soup_post.find(id='cph_content_PnShowResult')
        if not result_div:
            print("找不到查詢結果區塊。")
            return pd.DataFrame()

        result_table = result_div.find('table')
        if not result_table:
            print("找不到查詢結果表格。")
            return pd.DataFrame()

        # --- 步驟 5: 提取資料並存入 List ---
        data_rows = []
        rows = result_table.find_all('tr')[1:]

        for i in range(0, len(rows), 2):
            main_row = rows[i]
            cols = main_row.find_all('td')

            case_number_tag = cols[2].find('a')
            if case_number_tag:
                # 組合完整的下載連結
                download_link = url.rsplit(
                    '/', 1)[0] + '/' + case_number_tag['href'].strip()
            else:
                continue  # 如果沒有連結，跳過此筆

            data_rows.append({
                '評議字號': case_number_tag.text.strip(),
                '評議決定日期': cols[3].text.strip(),
                '爭議類型': cols[4].text.strip(),
                '下載連結': download_link,
            })

        # --- 步驟 6: 建立 Pandas DataFrame ---
        return pd.DataFrame(data_rows)

    except requests.exceptions.RequestException as e:
        print(f"網路請求錯誤: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"處理過程中發生錯誤: {e}")
        return pd.DataFrame()


def sanitize_filename(filename: str) -> str:
    """
    清理檔案名稱，移除不合法的字元。

    Args:
        filename (str): 原始檔案名稱。

    Returns:
        str: 清理過後的檔案名稱。
    """
    return re.sub(r'[\\/*?:"<>|]', "", filename)


def download_articles_as_pdf(session: requests.Session, df: pd.DataFrame, output_dir: str):
    """
    遍歷 DataFrame，直接下載連結中的 PDF 檔案並儲存。

    Args:
        session (requests.Session): 用於保持連線狀態的 Session 物件。
        df (pd.DataFrame): 包含案件資訊的 DataFrame。
        output_dir (str): PDF 檔案的儲存目錄。
    """
    if df.empty:
        print("沒有可下載的資料。")
        return

    # 確保輸出目錄存在
    os.makedirs(output_dir, exist_ok=True)
    print(f"PDF 將儲存至: {os.path.abspath(output_dir)}")

    total_files = len(df)
    for index, row in df.iterrows():
        try:
            # --- 步驟 1: 建立 PDF 檔案名稱並檢查是否已存在 ---
            seq_num = f"{index + 1:03d}"
            case_num_sanitized = sanitize_filename(row['評議字號'])
            dispute_type_sanitized = sanitize_filename(row['爭議類型'])

            pdf_filename = f"{seq_num}_{case_num_sanitized}_{dispute_type_sanitized}.pdf"
            pdf_filepath = os.path.join(output_dir, pdf_filename)

            if os.path.exists(pdf_filepath):
                print(f"[{index + 1}/{total_files}] 已存在，跳過: {pdf_filename}")
                continue

            # --- 步驟 2: 溫和爬蟲機制 ---
            sleep_time = random.uniform(1.0, 2.5)
            print(
                f"[{index + 1}/{total_files}] 準備下載: {pdf_filename} (等待 {sleep_time:.2f} 秒)")
            time.sleep(sleep_time)

            # --- 步驟 3: 下載 PDF 檔案 ---
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            pdf_res = session.get(row['下載連結'], headers=headers, timeout=30)
            pdf_res.raise_for_status()

            # --- 步驟 4: 將二進位內容寫入檔案 ---
            # 使用 'wb' (write binary) 模式來寫入 PDF 檔案
            with open(pdf_filepath, 'wb') as f:
                f.write(pdf_res.content)

            print(f"----> 成功儲存: {pdf_filename}")

        except requests.exceptions.RequestException as e:
            print(f"----! 下載失敗: {row['評議字號']}，錯誤: {e}")
        except Exception as e:
            print(f"----! 儲存檔案時發生錯誤: {row['評議字號']}，錯誤: {e}")


if __name__ == '__main__':
    # --- 設定區 ---
    target_url = 'https://ods.foi.org.tw/'
    search_keyword = 'prp 韌帶 或 prp半月板'
    output_directory = '金融評議決定書_PRP'  # 將資料夾名稱改得更具體

    # 建立共用的 Session
    with requests.Session() as session:
        # 執行爬蟲函數，取得案件列表
        print("開始爬取案件列表...")
        df_results = crawl_foi_decisions(session, target_url, search_keyword)

        if not df_results.empty:
            print(f"成功爬取到 {len(df_results)} 筆案件列表。")

            # 執行下載 PDF 的函數
            download_articles_as_pdf(session, df_results, output_directory)

            print("\n所有任務完成。")
        else:
            print("未能爬取到任何案件列表，程式結束。")

開始爬取案件列表...
成功爬取到 100 筆案件列表。
PDF 將儲存至: c:\Users\user\Documents\GitHub\judgment_python\金融評議決定書_PRP
[1/100] 準備下載: 001_113年評字第005616號_違反告知義務.pdf (等待 1.39 秒)
----> 成功儲存: 001_113年評字第005616號_違反告知義務.pdf
[2/100] 準備下載: 002_114年評字第000223號_投保時已患疾病或在妊娠中.pdf (等待 2.47 秒)
----> 成功儲存: 002_114年評字第000223號_投保時已患疾病或在妊娠中.pdf
[3/100] 準備下載: 003_114年評字第000268號_必要性醫療.pdf (等待 1.01 秒)
----> 成功儲存: 003_114年評字第000268號_必要性醫療.pdf
[4/100] 準備下載: 004_114年評字第000635號_必要性醫療.pdf (等待 1.12 秒)
----> 成功儲存: 004_114年評字第000635號_必要性醫療.pdf
[5/100] 準備下載: 005_114年評字第000193號_必要性醫療.pdf (等待 1.19 秒)
----> 成功儲存: 005_114年評字第000193號_必要性醫療.pdf
[6/100] 準備下載: 006_113年評字第005556號_失能或豁免保費體況認定.pdf (等待 2.25 秒)
----> 成功儲存: 006_113年評字第005556號_失能或豁免保費體況認定.pdf
[7/100] 準備下載: 007_113年評字第004478號_違反告知義務.pdf (等待 1.20 秒)
----> 成功儲存: 007_113年評字第004478號_違反告知義務.pdf
[8/100] 準備下載: 008_113年評字第004593號_違反告知義務.pdf (等待 1.75 秒)
----> 成功儲存: 008_113年評字第004593號_違反告知義務.pdf
[9/100] 準備下載: 009_113年評字第004982號_契約效力爭議.pdf (等待 1.19 秒)
----> 成功儲存: 009_113年評字第004982號_契約效力爭議.pdf
[10/